# Functions for calculations

This notebook includes the functions to calculate

1. Probability of short-term (one-year) GMST reduction/increase greater than the temperature thresholds between 0.1ºC and 0.9ºC by 2100

Main functions:
- calc_prob_1year_cooling
- calc_prob_2year_cooling
- calc_prob_1year_warming
- calc_prob_2year_warming

2. Decadal GMST trends using stochastic and historically-averaged forcing

Main function: calc_decadal_trend


---------------

Written by May Chim

Last updated: 9 August 2024

---------------

## Import packages for functions

In [1]:
import xarray as xr
import os
import numpy as np
from scipy import stats
import pandas as pd

## Probability of short-term (1-year and 2-year) GMST increase and reduction

In [2]:
# Note: 

    # The functions below are used to calculate the probability of at least one occurrence of 
    # short-term (1-year and 2-year) GMST increase and reduction by 2100.
    # See Figure 3a, Figure S3 and Figure S4.
    
    # To use the below function, you need to indicate:
    
    # 1. df (df2 or dn_internal)
        # df2: with stochastic volcanic forcing and internal variability,
        # dn_internal: with internal variability and no stochastic forcing)
        
    # 2. ssp index (0 or 1 or 2)
        # 0: ssp119, 1:ssp245, 2:ssp585
        
        
    # Datasets used:

    # df2 = xr.open_dataset('stochastic_volcanoes_stochastic_climate.nc')
    # dn_internal = xr.open_dataset('no_future_volcanoes_stochastic_climate.nc') 

        
    # Example:
    
    # Probability of at least one occurrence of 1-year GMST reduction for SSP1-1.9
    # using stochastic volcanic forcing
    # = calc_prob_1year_cooling(df2, 0)
    
    # Probability of at least one occurrence of 1-year GMST reduction for SSP1-1.9
    # using historically-averaged volcanic forcing
    # = calc_prob_1year_cooling(dn_internal, 0)

In [3]:
def calc_prob_1year_cooling(df, ssp):
    
    # Function to calculate the probability of short term (1-year) GMST reduction 
    # greater than temperature thresholds by 2100 (Figure 3a)
        
    thres = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9]
    prob_occurrences = []
    
    for threshold in thres:
        
        print('Threshold: -', threshold)
        total_count_future = []
        
        for future in range(1000):
            
            count_num = []
            
            for t in range(86):
                
                count = 0
                
                if ((df.temperature[165+t+1,ssp,future]-df.temperature[165+t,ssp,future]) < -threshold): 
                    count = count + 1
                    
                count_num.append(count)
                
            count_per_year = np.sum(count_num)
            total_count_future.append(count_per_year)

        count_scenario = sum(1 for x in total_count_future if x > 0)
        print('Probability of at least one occurrence of short-term cooling >', threshold,'ºC', count_scenario/ 1000 * 100)
        
        prob_occurrences.append(count_scenario / 1000 * 100)
        
    return prob_occurrences


def calc_prob_2year_cooling(df, ssp):
    
    # Function to calculate the probability of short term (2-year) GMST reduction 
    # greater than temperature thresholds by 2100 (Figure S3)

    
    thres = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]#, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9]
    prob_occurrences = []

    for threshold in thres:
        print('Threshold: -', threshold)

        total_count_future = []

        for future in range(1000):
            count_num = []
            for t in range(86-1):
                
                count = 0
                
                t1 = df.temperature[165+t+1,ssp,future] - df.temperature[165+t,ssp,future]
                t2 = df.temperature[165+t+2,ssp,future] - df.temperature[165+t+1,ssp,future]
                mean = (t1+t2)/2

                if (mean < -threshold) & (t1 < -0.1) & (t2 < -0.1):
                    count = count + 1
                    
                count_num.append(count)
                
            count_per_year = np.sum(count_num)
            total_count_future.append(count_per_year)

        count_each_scenario = sum(1 for x in total_count_future if x > 0)
        prob_occurrences.append(count_each_scenario / 1000 * 100)
        
        print('Probability of at least one occurrence of 2-year short-term cooling >', 
              threshold,'ºC', count_each_scenario/ 1000 * 100)
        
    return prob_occurrences

In [4]:
def calc_prob_1year_warming(df, ssp):
    
    # Function to calculate the probability of short term (1-year) GMST increase 
    # greater than temperature thresholds by 2100 (Figure S4a)

    
    thres = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9]
    prob_occurrences = []
    
    for threshold in thres:
        
        print('Threshold: ', threshold)
        total_count_future = []
        
        for future in range(1000):
            
            count_num = []
            
            for t in range(86):
                
                count = 0
                
                if ((df.temperature[165+t+1,ssp,future]-df.temperature[165+t,ssp,future]) > threshold): 
                    count = count + 1
                    
                count_num.append(count)
                
            count_per_year = np.sum(count_num)
            total_count_future.append(count_per_year)

        count_scenario = sum(1 for x in total_count_future if x > 0)
        print('Probability of at least one occurrence of 1-year GMST increase >', threshold,'ºC', count_scenario/ 1000 * 100)
        
        prob_occurrences.append(count_scenario / 1000 * 100)
        
    return prob_occurrences

def calc_prob_2year_warming(df, ssp):
    
    # Function to calculate the probability of short term (2-year) GMST increase 
    # greater than temperature thresholds by 2100 (Figure S4b)

    
    thres = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9]
    prob_occurrences = []

    for threshold in thres:
        print('Threshold: ', threshold)

        total_count_future = []

        for future in range(1000):
            count_num = []
            for t in range(86-1):
                
                count = 0
                
                t1 = df.temperature[165+t+1,ssp,future] - df.temperature[165+t,ssp,future]
                t2 = df.temperature[165+t+2,ssp,future] - df.temperature[165+t+1,ssp,future]
                mean = (t1+t2)/2

                if (mean > threshold) & (t1 > 0.1) & (t2 > 0.1):
                    count = count + 1
                    
                count_num.append(count)
                
            count_per_year = np.sum(count_num)
            total_count_future.append(count_per_year)

        count_scenario = sum(1 for x in total_count_future if x > 0)
        prob_occurrences.append(count_scenario / 1000 * 100)
        
        print('Probability of at least one occurrence of 1-year GMST increase >', 
              threshold,'ºC', count_scenario/ 1000 * 100)
        
    return prob_occurrences

## Decadal GMST trends

In [5]:
# Note: 

    # The functions below are used to calculate the decadal GMST trends using a 10-year moving window.
    # See Figure 3b and Figure S5.
    
    # To use the below function, you need to indicate:
    
    # 1. df (df2 or dn_internal)
        # df2: with stochastic volcanic forcing and internal variability,
        # dn_internal: with internal variability and no stochastic forcing)       
        
    # Datasets used:

    # df2 = xr.open_dataset('stochastic_volcanoes_stochastic_climate.nc')
    # dn_internal = xr.open_dataset('no_future_volcanoes_stochastic_climate.nc') 
      
    # Example:
    
    # Decadal trends for the simulation using stochastic volcanic forcing
    # decadal_trend_df2 = calc_decadal_trend(df2)
    
    # Decadal trends for the simulation using historically-averaged volcanic forcing
    # decadal_trend_dn = calc_decadal_trend(dn_internal)

In [6]:
def apply_linear_regression(group):
    
    # Function to apply linear regression to calculate decadal trend
    
    slope = xr.full_like(group.isel(timebounds=0), fill_value=np.nan, dtype=float)
   
    for scenario in group.scenario:
        for config in group.config:
            y = group.sel(scenario=scenario, config=config)
            x = np.arange(len(y))
            
            slope_val, _, _, _, _ = stats.linregress(x, y)
            slope.loc[scenario, config] = slope_val

    return xr.DataArray(slope)

def calc_decadal_trend(da, time_range = 2092-2015+1, window_size=10):
    
    # Function to calculate decadal GMST trends using a 10-year moving window
    # Change time_range and window_size in the function input for a different moving window.
    
    concat_trend = []
    
    index = 165 #starting year: 2015
    
    for x in range(time_range):
        print(index)
        
        data = da.temperature[index:index+window_size,:,0:2]
        trend = apply_linear_regression(data)
        concat_trend.append(trend)
        index = index + 1

    new_time = np.arange(2015,2093,1)
    decadal_trend = xr.concat(concat_trend, dim = 'timebounds')
    decadal_trend['timebounds'] = new_time
    
    return decadal_trend